In [1]:
!pip install torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0
!pip install insightface==0.2.1 onnxruntime moviepy
!pip install onnxruntime-gpu
!nvidia-smi
!pip install googledrivedownloader
!pip install timm==0.5.4
import torch
import torchvision
import moviepy
import os
!pip install ultralytics
from ultralytics import YOLO
from PIL import Image
import subprocess


from google.colab import drive
drive.mount('/content/drive')

from google.colab import files
uploaded = files.upload()
user_face_filename = list(uploaded.keys())[0]
user_face_path = os.path.join(os.getcwd(), user_face_filename)

gender_model_path = '/content/drive/MyDrive/GenderModel_YOLOv10.pt'
gender_model = YOLO(gender_model_path)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 43.2 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.4.0+cu121
    Uninstalling torch-2.4.0+cu121:
      Successfully uninstalled torch-2.4.0+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.19.0+cu121
    Uninstalling torchvision-0.19.0+cu121:
      Successfully uninstalled torchvision-0.19.0+cu121
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.4.0+cu121
    Uninstalling torchaudio-2.4.0+cu121:
      Successfully uninstalled torchaudio-2.4.0+cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 85.2 

Saving IMG_1746.jpeg to IMG_1746.jpeg


In [2]:
def DetectGender(user_face_path):
    image = Image.open(user_face_path)
    # Perform prediction
    results = gender_model.predict(image, verbose=False)
    confidence_threshold = 0.5
    prediction_made = False
    detected_gender = None

    # Iterate through results
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls)
            confidence = box.conf.item()
            # Output based on confidence threshold
            if confidence >= confidence_threshold:
                prediction_made = True  # A valid prediction is made
                if class_id == 1:  # Female with confidence > 0.5
                    detected_gender = 'female'
                elif class_id == 0:  # Male with confidence > 0.5
                    detected_gender = 'male'

    if not prediction_made:
        print("No confident prediction found. Please upload another image.")
        return None
    else:
        print(f"Detected gender: {detected_gender}")
        return detected_gender
print(DetectGender(user_face_path))


def setup_simswap():
    if not os.path.exists("SimSwap"):
        os.system("git clone https://github.com/neuralchen/SimSwap")
    else:
        print("SimSwap repository already exists.")


def show_videos(gender):
    if gender == 'male':
        video_options = os.listdir(male_videos)
    else:
        video_options = os.listdir(female_videos)

    selected_video = user_select_video(video_options, gender)
    return selected_video


def user_select_video(video_options, gender):
    if gender == 'male':
        folder_path = male_videos
    else:
        folder_path = female_videos

    # Filter out image files by only allowing video files (assuming .mp4 is the video extension)
    video_files = [video for video in video_options if video.endswith('.mp4')]

    for idx, video in enumerate(video_files):
        print(f"{idx+1}: {video}")
    choice = int(input("Select a video by number: ")) - 1
    selected_video = os.path.join(folder_path, video_files[choice])
    return selected_video


def run_simswap_from_selection(user_face_path, selected_video, checkpoint_dir):
    # Find the actor's face associated with the selected video
    actor_face = video_to_actor_map[selected_video]

    output_video_path = "/content/output_video.mp4"

    command = f"""
    python test_video_swapspecific.py --no_simswaplogo --crop_size 224 --use_mask \\
    --pic_specific_path "{actor_face}" \\
    --name people \\
    --Arc_path arcface_model/arcface_checkpoint.tar \\
    --pic_a_path "{user_face_path}" \\
    --video_path "{selected_video}" \\
    --checkpoints_dir "{checkpoint_dir}" \\
    --output_path "{output_video_path}" \\
    --temp_path ./temp_results
    """

    # Execute the command in a subprocess
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()

    if process.returncode == 0:
        print(f"SimSwap completed successfully! Output video saved to {output_video_path}")
    else:
        print(f"Error occurred during SimSwap execution: {stderr}")


male_videos = "/content/drive/MyDrive/ZAKA/Capstone Project/Movie Scenes/Scenes/male"
female_videos = "/content/drive/MyDrive/ZAKA/Capstone Project/Movie Scenes/Scenes/female"


video_to_actor_map = {
    "/content/drive/MyDrive/ZAKA/Capstone Project/Movie Scenes/Scenes/female/Maguy.mp4": "/content/drive/MyDrive/ZAKA/Capstone Project/Movie Scenes/Scenes/female/julia.png",
    "/content/drive/MyDrive/ZAKA/Capstone Project/Movie Scenes/Scenes/female/May ez dine.mp4": "/content/drive/MyDrive/ZAKA/Capstone Project/Movie Scenes/Scenes/female/MayEzDine.png",
    "/content/drive/MyDrive/ZAKA/Capstone Project/Movie Scenes/Scenes/female/Nadine Njem.mp4": "/content/drive/MyDrive/ZAKA/Capstone Project/Movie Scenes/Scenes/female/Nadine.png",
    "/content/drive/MyDrive/ZAKA/Capstone Project/Movie Scenes/Scenes/female/Samia al jazaeri.mp4": "/content/drive/MyDrive/ZAKA/Capstone Project/Movie Scenes/Scenes/female/samia.png",
    "/content/drive/MyDrive/ZAKA/Capstone Project/Movie Scenes/Scenes/female/Shokran.mp4": "/content/drive/MyDrive/ZAKA/Capstone Project/Movie Scenes/Scenes/female/Torfa.png",
    "/content/drive/MyDrive/ZAKA/Capstone Project/Movie Scenes/Scenes/male/Ahmad Helmi.mp4": "/content/drive/MyDrive/ZAKA/Capstone Project/Movie Scenes/Scenes/male/XLARGE.png",
    "/content/drive/MyDrive/ZAKA/Capstone Project/Movie Scenes/Scenes/male/Joude.mp4": "/content/drive/MyDrive/ZAKA/Capstone Project/Movie Scenes/Scenes/male/Day3aDay3a.png",
    "/content/drive/MyDrive/ZAKA/Capstone Project/Movie Scenes/Scenes/male/Moetassem al nahar.mp4": "/content/drive/MyDrive/ZAKA/Capstone Project/Movie Scenes/Scenes/male/Moetassem.png",
    "/content/drive/MyDrive/ZAKA/Capstone Project/Movie Scenes/Scenes/male/Taym AL Hassan.mp4": "/content/drive/MyDrive/ZAKA/Capstone Project/Movie Scenes/Scenes/male/TaymAlHassan.png",
    "/content/drive/MyDrive/ZAKA/Capstone Project/Movie Scenes/Scenes/male/Mohamad Henedi.mp4": "/content/drive/MyDrive/ZAKA/Capstone Project/Movie Scenes/Scenes/male/julia.png"

}


Detected gender: female
female


/usr/local/lib/python3.10/dist-packages/ultralytics/nn/modules/head.py:154: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return torch.cat([boxes[i, index // nc], scores[..., None], (index % nc)[..., None].float()], dim=-1)


In [3]:
setup_simswap() #setting up simswap environment
!mkdir -p SimSwap/insightface_func/models
!mkdir -p SimSwap/arcface_model
!gdown --id 1TLNdIufzwesDbyr_nVTR7Zrx9oRHLM_N -O SimSwap/arcface_model/arcface_checkpoint.tar

!cp /content/drive/MyDrive/antelope.zip* SimSwap/insightface_func/models/
!unzip ./SimSwap/insightface_func/models/antelope.zip -d ./SimSwap/insightface_func/models/
!mkdir -p SimSwap/checkpoints
!gdown --id 1PXkRiBUYbu1xWpQyDEJvGKeqqUFthJcI -O checkpoints.zip
!unzip checkpoints.zip -d SimSwap/
%cd SimSwap

!mkdir -p ./parsing_model/checkpoint
!cp /content/drive/MyDrive/79999_iter.pth* ./parsing_model/checkpoint/

!mkdir -p ./checkpoints/people
!mv people/latest_net_G.pth checkpoints/people/


/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1TLNdIufzwesDbyr_nVTR7Zrx9oRHLM_N
From (redirected): https://drive.google.com/uc?id=1TLNdIufzwesDbyr_nVTR7Zrx9oRHLM_N&confirm=t&uuid=de24738c-5c74-40de-b9af-939f791b308f
To: /content/SimSwap/arcface_model/arcface_checkpoint.tar
100% 209M/209M [00:02<00:00, 71.3MB/s]
Archive:  ./SimSwap/insightface_func/models/antelope.zip
   creating: ./SimSwap/insightface_func/models/antelope/
  inflating: ./SimSwap/insightface_func/models/antelope/glintr100.onnx  
  inflating: ./SimSwap/insightface_func/models/antelope/scrfd_10g_bnkps.onnx  
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymo

In [6]:
!ls ./checkpoints/people

latest_net_G.pth


In [10]:
gender = DetectGender(user_face_path)
selected_video = show_videos(gender)

# Now, run SimSwap with the selected video and corresponding actor's face
run_simswap_from_selection(user_face_path, selected_video, "./checkpoints")

Detected gender: female
1: Maguy.mp4
2: Shokran.mp4
3: May ez dine.mp4
4: Nadine Njem.mp4
5: Samia al jazaeri.mp4
Select a video by number: 5
SimSwap completed successfully! Output video saved to /content/output_video.mp4
